In [1]:
.libPaths('/disk1/wenqing/anaconda3/envs/wq_R4.2/lib/R/library')

library(IRkernel)
library(jsonlite)
library(repr)

In [2]:
ctr_sample_iso_IR <- read.table("/disk1/wenqing/tmp_data/ASD/ctr_male_pfc/iso_retained_intron/ctr_iso_IR.txt",header=F)
ctr_sample_iso_no_IR <- read.table("/disk1/wenqing/tmp_data/ASD/ctr_male_pfc/iso_retained_intron/ctr_iso_no_IR.txt",header=F)

In [3]:
head(ctr_sample_iso_IR)

,V1
,<chr>
1,ENST00000595014.1_3
2,ENST00000472360.1_3
3,ENST00000495442.1_3
4,ENST00000495709.1_3
5,ENST00000546393.7_4
6,ENST00000552876.5_3


In [5]:
#调取两组iso的exon边界，构建剪接供受体位点
exon_region <- read.csv("/disk1/wenqing/tmp_data/ASD/switchList_exon.csv")
dim(exon_region)

[1] 926332      9

In [6]:
iso_IR_exon_region <- exon_region[exon_region$isoform_id %in% ctr_sample_iso_IR$V1,]
iso_no_IR_exon_region <- exon_region[exon_region$isoform_id %in% ctr_sample_iso_no_IR$V1,]

In [7]:
head(iso_IR_exon_region)

,X,seqnames,start,end,width,strand,isoform_id,gene_id,gene_name
,<int>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<chr>
125,125,chr1,786728,787490,763,+,ENST00000425657.1_1,LINC01128,LINC01128
132,132,chr1,788051,788132,82,+,ENST00000425657.1_1,LINC01128,LINC01128
184,184,chr1,898932,899388,457,+,ENST00000481067.1_1,KLHL17,KLHL17
187,187,chr1,899487,899544,58,+,ENST00000481067.1_1,KLHL17,KLHL17
220,220,chr1,970247,970704,458,+,ENST00000469403.1_1,AGRN,AGRN
223,223,chr1,976045,976260,216,+,ENST00000469403.1_1,AGRN,AGRN


In [8]:
iso_IR_exon_region <- iso_IR_exon_region[,c(2:9)]
iso_no_IR_exon_region <- iso_no_IR_exon_region[,c(2:9)]
write.table(iso_IR_exon_region,"/disk1/wenqing/tmp_data/ASD/ctr_iso_IR_exon_region.txt",row.names=F,quote=F)
write.table(iso_no_IR_exon_region,"/disk1/wenqing/tmp_data/ASD/ctr_iso_no_IR_exon_region.txt",row.names=F,quote=F)

In [9]:
head(iso_IR_exon_region)

,seqnames,start,end,width,strand,isoform_id,gene_id,gene_name
,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<chr>
125,chr1,786728,787490,763,+,ENST00000425657.1_1,LINC01128,LINC01128
132,chr1,788051,788132,82,+,ENST00000425657.1_1,LINC01128,LINC01128
184,chr1,898932,899388,457,+,ENST00000481067.1_1,KLHL17,KLHL17
187,chr1,899487,899544,58,+,ENST00000481067.1_1,KLHL17,KLHL17
220,chr1,970247,970704,458,+,ENST00000469403.1_1,AGRN,AGRN
223,chr1,976045,976260,216,+,ENST00000469403.1_1,AGRN,AGRN


In [10]:
start_exd <- iso_IR_exon_region$start-5
end_exd <- iso_IR_exon_region$end+5
iso_IR_exon_region_exd <- iso_IR_exon_region
iso_IR_exon_region_exd$start <- start_exd
iso_IR_exon_region_exd$end <- end_exd

In [11]:
no_IR_start_exd <- iso_no_IR_exon_region$start-5
no_IR_end_exd <- iso_no_IR_exon_region$end+5
iso_no_IR_exon_region_exd <- iso_no_IR_exon_region
iso_no_IR_exon_region_exd$start <- no_IR_start_exd
iso_no_IR_exon_region_exd$end <- no_IR_end_exd

In [12]:
write.table(iso_IR_exon_region_exd,"/disk1/wenqing/tmp_data/ASD/ctr_iso_IR_exon_region_exd.txt",row.names=F,quote=F)
write.table(iso_no_IR_exon_region_exd,"/disk1/wenqing/tmp_data/ASD/ctr_iso_no_IR_exon_region_exd.txt",row.names=F,quote=F)

In [13]:
#res是否落在可变剪接的供受体位点上
#构建供受体位点上下游5bp的潜在res位点
donor_sites_2 <- paste0(iso_IR_exon_region$seqnames,"_",as.character(as.integer(iso_IR_exon_region$end)+2))
donor_sites_1 <- paste0(iso_IR_exon_region$seqnames,"_",as.character(as.integer(iso_IR_exon_region$end)+1))
donor_sites_3 <- paste0(iso_IR_exon_region$seqnames,"_",as.character(as.integer(iso_IR_exon_region$end)+3))
donor_sites_4 <- paste0(iso_IR_exon_region$seqnames,"_",as.character(as.integer(iso_IR_exon_region$end)+4))
donor_sites_5 <- paste0(iso_IR_exon_region$seqnames,"_",as.character(as.integer(iso_IR_exon_region$end)+5))
donor_sites_2_neg <- paste0(iso_IR_exon_region$seqnames,"_",as.character(as.integer(iso_IR_exon_region$end)-2))
donor_sites_1_neg <- paste0(iso_IR_exon_region$seqnames,"_",as.character(as.integer(iso_IR_exon_region$end)-1))
donor_sites_3_neg <- paste0(iso_IR_exon_region$seqnames,"_",as.character(as.integer(iso_IR_exon_region$end)-3))
donor_sites_4_neg <- paste0(iso_IR_exon_region$seqnames,"_",as.character(as.integer(iso_IR_exon_region$end)-4))
donor_sites_5_neg <- paste0(iso_IR_exon_region$seqnames,"_",as.character(as.integer(iso_IR_exon_region$end)-5))

acceptor_sites_2 <- paste0(iso_IR_exon_region$seqnames,"_",as.character(as.integer(iso_IR_exon_region$start)-2))
acceptor_sites_1 <- paste0(iso_IR_exon_region$seqnames,"_",as.character(as.integer(iso_IR_exon_region$start)-1))
acceptor_sites_3 <- paste0(iso_IR_exon_region$seqnames,"_",as.character(as.integer(iso_IR_exon_region$start)-3))
acceptor_sites_4 <- paste0(iso_IR_exon_region$seqnames,"_",as.character(as.integer(iso_IR_exon_region$start)-4))
acceptor_sites_5 <- paste0(iso_IR_exon_region$seqnames,"_",as.character(as.integer(iso_IR_exon_region$start)-5))

acceptor_sites_2_pos <- paste0(iso_IR_exon_region$seqnames,"_",as.character(as.integer(iso_IR_exon_region$start)+2))
acceptor_sites_1_pos <- paste0(iso_IR_exon_region$seqnames,"_",as.character(as.integer(iso_IR_exon_region$start)+1))
acceptor_sites_3_pos <- paste0(iso_IR_exon_region$seqnames,"_",as.character(as.integer(iso_IR_exon_region$start)+3))
acceptor_sites_4_pos <- paste0(iso_IR_exon_region$seqnames,"_",as.character(as.integer(iso_IR_exon_region$start)+4))
acceptor_sites_5_pos <- paste0(iso_IR_exon_region$seqnames,"_",as.character(as.integer(iso_IR_exon_region$start)+5))


all_possible_sites <- c(donor_sites_1,donor_sites_2,donor_sites_3,donor_sites_4,donor_sites_5,acceptor_sites_1,acceptor_sites_2,acceptor_sites_3,acceptor_sites_4,acceptor_sites_5,
                        donor_sites_1_neg,donor_sites_2_neg,donor_sites_3_neg,donor_sites_4_neg,donor_sites_5_neg,
                        acceptor_sites_1_pos,acceptor_sites_2_pos,acceptor_sites_3_pos,acceptor_sites_4_pos,acceptor_sites_5_pos)

donor_sites_info <- c(rep("donor_ds_1_G",times=length(iso_IR_exon_region$seqnames)),rep("donor_ds_2_T",times=length(iso_IR_exon_region$seqnames)),rep("donor_ds_3_A_G",times=length(iso_IR_exon_region$seqnames)),rep("donor_ds_4_A",times=length(iso_IR_exon_region$seqnames)),rep("donor_ds_5_G",times=length(iso_IR_exon_region$seqnames)),
                        rep("donor_us_1",times=length(iso_IR_exon_region$seqnames)),rep("donor_us_2",times=length(iso_IR_exon_region$seqnames)),rep("donor_us_3",times=length(iso_IR_exon_region$seqnames)),rep("donor_us_4",times=length(iso_IR_exon_region$seqnames)),rep("donor_us_5",times=length(iso_IR_exon_region$seqnames)))
acceptor_sites_info <- c(rep("acceptor_us_1_G",times=length(iso_IR_exon_region$seqnames)),rep("acceptor_us_2_A",times=length(iso_IR_exon_region$seqnames)),rep("acceptor_us_3_C_T",times=length(iso_IR_exon_region$seqnames)),rep("acceptor_us_4_N",times=length(iso_IR_exon_region$seqnames)),rep("acceptor_us_5_C_T",times=length(iso_IR_exon_region$seqnames)),
                        rep("acceptor_ds_1",times=length(iso_IR_exon_region$seqnames)),rep("acceptor_ds_2",times=length(iso_IR_exon_region$seqnames)),rep("acceptor_ds_3",times=length(iso_IR_exon_region$seqnames)),rep("acceptor_ds_4",times=length(iso_IR_exon_region$seqnames)),rep("acceptor_ds_5",times=length(iso_IR_exon_region$seqnames)))

all_sites_info <- c(donor_sites_info,acceptor_sites_info)

isoform_id <- rep(iso_IR_exon_region$isoform_id,times=20)

In [14]:
all_sites <- as.data.frame(cbind(all_possible_sites,all_sites_info,isoform_id))

In [15]:
head(all_sites)

,all_possible_sites,all_sites_info,isoform_id
,<chr>,<chr>,<chr>
1,chr1_787491,donor_ds_1_G,ENST00000425657.1_1
2,chr1_788133,donor_ds_1_G,ENST00000425657.1_1
3,chr1_899389,donor_ds_1_G,ENST00000481067.1_1
4,chr1_899545,donor_ds_1_G,ENST00000481067.1_1
5,chr1_970705,donor_ds_1_G,ENST00000469403.1_1
6,chr1_976261,donor_ds_1_G,ENST00000469403.1_1


In [16]:
dim(all_sites)
colnames(all_sites) <- c("site","info","iso_id")
head(all_sites)


[1] 2154860       3

,site,info,iso_id
,<chr>,<chr>,<chr>
1,chr1_787491,donor_ds_1_G,ENST00000425657.1_1
2,chr1_788133,donor_ds_1_G,ENST00000425657.1_1
3,chr1_899389,donor_ds_1_G,ENST00000481067.1_1
4,chr1_899545,donor_ds_1_G,ENST00000481067.1_1
5,chr1_970705,donor_ds_1_G,ENST00000469403.1_1
6,chr1_976261,donor_ds_1_G,ENST00000469403.1_1


In [17]:
uniq_all_sites <- all_sites[!duplicated(all_sites$site),]
dim(uniq_all_sites)

rownames(uniq_all_sites) <- uniq_all_sites$site

[1] 1490917       3

In [19]:
#head(all_possible_sites)
#all.res.wizSampleInfo.new: 由ctr组中的23组成
all_res <- read.table("/disk1/wenqing/tmp_data/ASD/all.res.wizSampleInfo.new")

#ctr only
all_res <- all_res[all_res$V9=='20'|all_res$V9=='23'|all_res$V9=='56',]
head(all_res)

all_res$V10 <- paste0(all_res$V1,"_",all_res$V3)
head(all_res)

,V1,V2,V3,V4,V5,V6,V7,V8,V9
,<chr>,<int>,<int>,<chr>,<int>,<chr>,<chr>,<chr>,<int>
10353077,chr1,51693,51694,GA,1,-,AGCCTAACAATAAGCA,2:2,20
10353078,chr1,51715,51716,GA,1,-,AGCCTAACAATAAGCA,2:2,20
10353079,chr1,51715,51716,GA,1,-,GCCAAATTCTTTACGT,1:1,20
10353080,chr1,51728,51729,GA,1,-,GCCAAATTCTTTACGT,1:1,20
10353081,chr1,51738,51739,GA,1,-,AGCCTAACAATAAGCA,1:2,20
10353082,chr1,51744,51745,GA,1,-,AGCCTAACAATAAGCA,1:1,20


,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10
,<chr>,<int>,<int>,<chr>,<int>,<chr>,<chr>,<chr>,<int>,<chr>
10353077,chr1,51693,51694,GA,1,-,AGCCTAACAATAAGCA,2:2,20,chr1_51694
10353078,chr1,51715,51716,GA,1,-,AGCCTAACAATAAGCA,2:2,20,chr1_51716
10353079,chr1,51715,51716,GA,1,-,GCCAAATTCTTTACGT,1:1,20,chr1_51716
10353080,chr1,51728,51729,GA,1,-,GCCAAATTCTTTACGT,1:1,20,chr1_51729
10353081,chr1,51738,51739,GA,1,-,AGCCTAACAATAAGCA,1:2,20,chr1_51739
10353082,chr1,51744,51745,GA,1,-,AGCCTAACAATAAGCA,1:1,20,chr1_51745


In [20]:
write.table(all_res,"/disk1/wenqing/tmp_data/ASD/ctr_male_pfc/ctr_all.res",quote=F,col.names=F,row.names=F)

In [21]:
resInSpliceSites <- all_res[all_res$V10 %in% all_sites$site,]


In [22]:
resInSpliceSites$V11 <- "info"
resInSpliceSites$V11 <- uniq_all_sites[resInSpliceSites$V10,]$info
resInSpliceSites$V12 <- "iso_id"
resInSpliceSites$V12 <- uniq_all_sites[resInSpliceSites$V10,]$iso_id



In [23]:
head(resInSpliceSites)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12
,<chr>,<int>,<int>,<chr>,<int>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>
10355292,chr1,1419947,1419948,AG,1,-,TACCTTACATCATCCC,1:1,20,chr1_1419948,donor_us_1,ENST00000474481.1_1
10394882,chr1,21219604,21219605,TC,1,+,TGGACGCCAGACGTAG,1:1,20,chr1_21219605,acceptor_ds_1,ENST00000374933.3_3
10416512,chr1,29500007,29500008,TC,3,+,ATCATGGCAGATAATG,1:1,20,chr1_29500008,donor_ds_2_T,ENST00000634348.1_3
10422126,chr1,32696984,32696985,AG,1,-,CGTGTAATCAGTGTTG,1:1,20,chr1_32696985,donor_ds_4_A,ENST00000678306.1_1
10422127,chr1,32696993,32696994,AG,1,-,CGTGTAATCAGTGTTG,1:1,20,chr1_32696994,acceptor_us_1_G,ENST00000489353.6_2
10423805,chr1,33097659,33097660,TC,1,+,GTTTCTACAAGCGAGT,3:3,20,chr1_33097660,donor_ds_1_G,ENST00000467652.5_3


In [25]:
#计算as site有RES的iso数量，不去重
length(unique(resInSpliceSites$V10))

[1] 989

In [26]:
#发生在retained itron上的所有编辑位点
ctr_res_in_iso_IR_region <- read.table("/disk1/wenqing/tmp_data/ASD/ctr_male_pfc/iso_retained_intron/ctr_res_in_iso_IR_region_exd.txt")$V1
head(ctr_res_in_iso_IR_region)
length(setdiff(unique(ctr_res_in_iso_IR_region),unique(resInSpliceSites$V10)))

[1] "chr1_1419867" "chr1_1419888" "chr1_1419898" "chr1_1419948" "chr1_1418942"
[6] "chr1_1418966"

[1] 47786

In [ ]:
length(unique(iso_IR_exon_region$isoform_id))

[1] 21416

In [ ]:
length(unique(setdiff(iso_IR_exon_region$isoform_id,unique(resInSpliceSites$V12))))

[1] 20872

In [ ]:
resInInIsoIR_region <- read.table("/disk1/wenqing/tmp_data/ctr/iso_IR_RES.txt")


[1] 192849      2

In [ ]:
head(resInInIsoIR_region)


,V1,V2
,<chr>,<chr>
1,chr1_1554509,ENST00000477990.1_1
2,chr1_1554510,ENST00000477990.1_1
3,chr1_1554511,ENST00000477990.1_1
4,chr1_1554560,ENST00000477990.1_1
5,chr1_1554564,ENST00000477990.1_1
6,chr1_1554583,ENST00000477990.1_1


In [ ]:
resNotInSpliceSites <- resInInIsoIR_region[!(resInInIsoIR_region$V1 %in% resInSpliceSites$V10),]
dim(resNotInSpliceSites)
length(unique(resNotInSpliceSites$V2))

[1] 192634      2

[1] 2133

In [ ]:
#下面为未发生IR的iso

In [27]:
#res是否落在可变剪接的供受体位点上
#构建供受体位点上下游5bp的潜在res位点
donor_sites_2 <- paste0(iso_no_IR_exon_region$seqnames,"_",as.character(as.integer(iso_no_IR_exon_region$end)+2))
donor_sites_1 <- paste0(iso_no_IR_exon_region$seqnames,"_",as.character(as.integer(iso_no_IR_exon_region$end)+1))
donor_sites_3 <- paste0(iso_no_IR_exon_region$seqnames,"_",as.character(as.integer(iso_no_IR_exon_region$end)+3))
donor_sites_4 <- paste0(iso_no_IR_exon_region$seqnames,"_",as.character(as.integer(iso_no_IR_exon_region$end)+4))
donor_sites_5 <- paste0(iso_no_IR_exon_region$seqnames,"_",as.character(as.integer(iso_no_IR_exon_region$end)+5))
donor_sites_2_neg <- paste0(iso_no_IR_exon_region$seqnames,"_",as.character(as.integer(iso_no_IR_exon_region$end)-2))
donor_sites_1_neg <- paste0(iso_no_IR_exon_region$seqnames,"_",as.character(as.integer(iso_no_IR_exon_region$end)-1))
donor_sites_3_neg <- paste0(iso_no_IR_exon_region$seqnames,"_",as.character(as.integer(iso_no_IR_exon_region$end)-3))
donor_sites_4_neg <- paste0(iso_no_IR_exon_region$seqnames,"_",as.character(as.integer(iso_no_IR_exon_region$end)-4))
donor_sites_5_neg <- paste0(iso_no_IR_exon_region$seqnames,"_",as.character(as.integer(iso_no_IR_exon_region$end)-5))

acceptor_sites_2 <- paste0(iso_no_IR_exon_region$seqnames,"_",as.character(as.integer(iso_no_IR_exon_region$start)-2))
acceptor_sites_1 <- paste0(iso_no_IR_exon_region$seqnames,"_",as.character(as.integer(iso_no_IR_exon_region$start)-1))
acceptor_sites_3 <- paste0(iso_no_IR_exon_region$seqnames,"_",as.character(as.integer(iso_no_IR_exon_region$start)-3))
acceptor_sites_4 <- paste0(iso_no_IR_exon_region$seqnames,"_",as.character(as.integer(iso_no_IR_exon_region$start)-4))
acceptor_sites_5 <- paste0(iso_no_IR_exon_region$seqnames,"_",as.character(as.integer(iso_no_IR_exon_region$start)-5))

acceptor_sites_2_pos <- paste0(iso_no_IR_exon_region$seqnames,"_",as.character(as.integer(iso_no_IR_exon_region$start)+2))
acceptor_sites_1_pos <- paste0(iso_no_IR_exon_region$seqnames,"_",as.character(as.integer(iso_no_IR_exon_region$start)+1))
acceptor_sites_3_pos <- paste0(iso_no_IR_exon_region$seqnames,"_",as.character(as.integer(iso_no_IR_exon_region$start)+3))
acceptor_sites_4_pos <- paste0(iso_no_IR_exon_region$seqnames,"_",as.character(as.integer(iso_no_IR_exon_region$start)+4))
acceptor_sites_5_pos <- paste0(iso_no_IR_exon_region$seqnames,"_",as.character(as.integer(iso_no_IR_exon_region$start)+5))


all_possible_sites <- c(donor_sites_1,donor_sites_2,donor_sites_3,donor_sites_4,donor_sites_5,acceptor_sites_1,acceptor_sites_2,acceptor_sites_3,acceptor_sites_4,acceptor_sites_5,
                        donor_sites_1_neg,donor_sites_2_neg,donor_sites_3_neg,donor_sites_4_neg,donor_sites_5_neg,
                        acceptor_sites_1_pos,acceptor_sites_2_pos,acceptor_sites_3_pos,acceptor_sites_4_pos,acceptor_sites_5_pos)

donor_sites_info <- c(rep("donor_ds_1_G",times=length(iso_no_IR_exon_region$seqnames)),rep("donor_ds_2_T",times=length(iso_no_IR_exon_region$seqnames)),rep("donor_ds_3_A_G",times=length(iso_no_IR_exon_region$seqnames)),rep("donor_ds_4_A",times=length(iso_no_IR_exon_region$seqnames)),rep("donor_ds_5_G",times=length(iso_no_IR_exon_region$seqnames)),
                        rep("donor_us_1",times=length(iso_no_IR_exon_region$seqnames)),rep("donor_us_2",times=length(iso_no_IR_exon_region$seqnames)),rep("donor_us_3",times=length(iso_no_IR_exon_region$seqnames)),rep("donor_us_4",times=length(iso_no_IR_exon_region$seqnames)),rep("donor_us_5",times=length(iso_no_IR_exon_region$seqnames)))
acceptor_sites_info <- c(rep("acceptor_us_1_G",times=length(iso_no_IR_exon_region$seqnames)),rep("acceptor_us_2_A",times=length(iso_no_IR_exon_region$seqnames)),rep("acceptor_us_3_C_T",times=length(iso_no_IR_exon_region$seqnames)),rep("acceptor_us_4_N",times=length(iso_no_IR_exon_region$seqnames)),rep("acceptor_us_5_C_T",times=length(iso_no_IR_exon_region$seqnames)),
                        rep("acceptor_ds_1",times=length(iso_no_IR_exon_region$seqnames)),rep("acceptor_ds_2",times=length(iso_no_IR_exon_region$seqnames)),rep("acceptor_ds_3",times=length(iso_no_IR_exon_region$seqnames)),rep("acceptor_ds_4",times=length(iso_no_IR_exon_region$seqnames)),rep("acceptor_ds_5",times=length(iso_no_IR_exon_region$seqnames)))

all_sites_info <- c(donor_sites_info,acceptor_sites_info)

isoform_id <- rep(iso_no_IR_exon_region$isoform_id,times=20)

all_sites <- as.data.frame(cbind(all_possible_sites,all_sites_info,isoform_id))

In [28]:
dim(all_sites)
colnames(all_sites) <- c("site","info","iso_id")
head(all_sites)

uniq_all_sites <- all_sites[!duplicated(all_sites$site),]
dim(uniq_all_sites)

rownames(uniq_all_sites) <- uniq_all_sites$site

resInSpliceSites <- all_res[all_res$V10 %in% all_sites$site,]

resInSpliceSites$V11 <- "info"
resInSpliceSites$V11 <- uniq_all_sites[resInSpliceSites$V10,]$info
resInSpliceSites$V12 <- "iso_id"
resInSpliceSites$V12 <- uniq_all_sites[resInSpliceSites$V10,]$iso_id

dim(resInSpliceSites)


[1] 13887800        3

,site,info,iso_id
,<chr>,<chr>,<chr>
1,chr1_65434,donor_ds_1_G,ENST00000641515.2_5
2,chr1_65574,donor_ds_1_G,ENST00000641515.2_5
3,chr1_71586,donor_ds_1_G,ENST00000641515.2_5
4,chr1_134837,donor_ds_1_G,ENST00000442987.3_2
5,chr1_160691,donor_ds_1_G,ENST00000496488.1_5
6,chr1_161526,donor_ds_1_G,ENST00000496488.1_5


[1] 6661737       3

[1] 13420    12

In [29]:
length(unique(resInSpliceSites$V10))


[1] 5101

In [30]:
ctr_res_in_iso_no_IR_region <- read.table("/disk1/wenqing/tmp_data/ASD/ctr_male_pfc/iso_retained_intron/ctr_res_in_iso_no_IR_region_exd.txt")$V1
head(ctr_res_in_iso_no_IR_region)
length(setdiff(unique(ctr_res_in_iso_no_IR_region),unique(resInSpliceSites$V10)))

[1] "chr1_134210" "chr1_134216" "chr1_134232" "chr1_134241" "chr1_134245"
[6] "chr1_134247"

[1] 156380

In [ ]:
length(unique(setdiff(iso_no_IR_exon_region$isoform_id,unique(resInSpliceSites$V12))))

[1] 100265

In [ ]:
length(unique(iso_no_IR_exon_region$isoform_id))

[1] 102950

In [ ]:

resInInIsoIR_region <- read.table("/disk1/wenqing/tmp_data/ctr/iso_no_IR_RES.txt")
dim(resInInIsoIR_region)

resNotInSpliceSites <- resInInIsoIR_region[!(resInInIsoIR_region$V1 %in% resInSpliceSites$V10),]
dim(resNotInSpliceSites)
length(unique(resNotInSpliceSites$V2))

In [ ]:
length(unique(resInSpliceSites$V12))

[1] 1410

In [32]:
my_ka <- matrix(c(989,47786,5101,156380),nrow=2,byrow=T)
colnames(my_ka) <- c('0','1')

rownames(my_ka) <- c('0','1')
my_ka
chisq.test(my_ka)

,0,1
0,989,47786
1,5101,156380



	Pearson's Chi-squared test with Yates' continuity correction

data:  my_ka
X-squared = 170.03, df = 1, p-value < 2.2e-16


In [31]:
989/47786
5101/156380


[1] 0.02069644

[1] 0.03261926

In [ ]:
17

[1] TRUE